In [1]:
%load_ext rich

In [2]:
import math

import phoenix as px

import promptimus as pm

In [3]:
provider = pm.llms.OllamaProvider(model_name="phi4", base_url="http://lilan:11434/v1")

In [4]:
def power(a: float, b: float) -> float:
    """Calcuates the `a` in the pover of `b`"""
    return a**b


def factorial(a: int) -> int:
    """Calcuates the factorial (!) of `a`"""
    return math.factorial(a)


def multiply(a: float, b: float) -> float:
    """Multiplies `a` and `b`"""
    return a * b

In [ ]:
px.launch_app()

In [6]:
tool_calling = pm.modules.ToolCallingAgent([power, multiply, factorial]).with_provider(
    provider
)
pm.tracing.trace_phoenix(
    tool_calling,
    "agent",
    project_name="tool_calling_test",
    endpoint="http://localhost:4317",
)

2025-02-28 14:30:26.480 | DEBUG    | promptimus.tracing.phoenix:_trace_module:124 - Wrapping `agent.tools`
2025-02-28 14:30:26.480 | DEBUG    | promptimus.tracing.phoenix:_trace_module:119 - Wrapping `agent.tools.power`
2025-02-28 14:30:26.481 | DEBUG    | promptimus.tracing.phoenix:_trace_module:119 - Wrapping `agent.tools.multiply`
2025-02-28 14:30:26.481 | DEBUG    | promptimus.tracing.phoenix:_trace_module:119 - Wrapping `agent.tools.factorial`
2025-02-28 14:30:26.481 | DEBUG    | promptimus.tracing.phoenix:_trace_module:124 - Wrapping `agent.predictor`
2025-02-28 14:30:26.482 | DEBUG    | promptimus.tracing.phoenix:_trace_module:114 - Wrapping `agent.predictor.prompt`


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: tool_calling_test
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



In [7]:
await tool_calling.forward("What is twice the factorial of 3?")

Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='Twice the factorial of 3 is 12.')

In [17]:
class CustomRole(pm.Module):
    def __init__(self):
        super().__init__()
        self.prompt = pm.Prompt("Respond to user using provided documents")
        self.documents = ["doc1: About Elephants", "doc2: About Pythons"]
    async def forward(self, query: str) -> str:
        response = await self.prompt.forward([pm.Message(role=pm.MessageRole.USER, content=query), pm.Message(role="Documents", content=str(self.documents))])
        return response.content

In [18]:
rag_like = CustomRole().with_provider(
    provider
)
pm.tracing.trace_phoenix(
    rag_like,
    "test_rag",
    project_name="custom_role",
    endpoint="http://localhost:4317",
)

2025-02-28 14:34:42.612 | DEBUG    | promptimus.tracing.phoenix:_trace_module:114 - Wrapping `test_rag.prompt`


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: custom_role
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



In [19]:
await rag_like.forward("Whats's your docs?")

'Based on the documents provided:\n\n1. **Document 1** is about elephants. It likely contains information related to their physical characteristics, habitats, behaviors, and conservation status.\n\n2. **Document 2** focuses on pythons. This document probably covers details concerning types of pythons, their physiology, natural environments, behaviors, diet, and perhaps issues surrounding conservation or their impact on ecosystems.\n\nIf you have any specific questions about elephants or pythons based on these documents, feel free to ask!'